# Prepping Kaggle Test Dataframe, Part 2

In this notebook, I put the cleaned Kaggle test.csv dataframe (aka "draft1_cleaned_kaggle_test.csv" ) through the feature engineering I did with the training dataframe.

I've taken some of the EDA out, but otherwise I've left many of my notes from the original EDA and Feature Engineering notebook intact to make it easier to follow the workflow. 

In [114]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [115]:
df = pd.read_csv('datasets/draft1_cleaned_kaggle_test.csv')

In [116]:
df.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,4,2006,WD,0
1,2718,905108090,90,RL,0.0,9662,Pave,None,IR1,Lvl,...,0,0,No Pool,No Fence,No Addl Features,0,8,2006,WD,0


In [117]:
df.shape

(878, 81)

I'm returning here to double check things that may need further cleaning, to see if they were correct or not correct in the 'draft1_cleaned_train.csv' file

In [118]:
df['street'].value_counts()

Pave    873
Grvl      5
Name: street, dtype: int64

In [119]:
df['bsmt_qual'].value_counts()

TA             396
Gd             355
Ex              73
Fa              28
No Basement     25
Po               1
Name: bsmt_qual, dtype: int64

I found this `df.loc[(df.name == None) & (df.weight == 1), 'name'] = 'defaut'` at [this site](https://stackoverflow.com/questions/44641577/how-to-change-value-for-rows-that-meet-specific-conditions-in-pandas) and modified it to make the change I want.

In [120]:
df['has_bsmt'] = 0

df.loc[(df['bsmt_qual'] != 'No Basement'), 'has_bsmt'] = 1

In [121]:
df['has_bsmt'].value_counts()

1    853
0     25
Name: has_bsmt, dtype: int64

In [122]:
df['garage_qual'].value_counts()

TA           782
No Garage     45
Fa            42
Gd             6
Po             3
Name: garage_qual, dtype: int64

In [123]:
df['has_garage'] = 0

df.loc[(df['garage_qual'] != 'No Garage'), 'has_garage']=1

In [124]:
df['has_garage'].value_counts()

1    833
0     45
Name: has_garage, dtype: int64

In [125]:
df['mas_vnr_type'].value_counts()

None       535
BrkFace    250
Stone       80
BrkCmn      12
CBlock       1
Name: mas_vnr_type, dtype: int64

In [126]:
df['has_mas_vnr'] = 0

df.loc[(df['mas_vnr_type'] != 'None'), 'has_mas_vnr'] = 1

In [127]:
df['has_mas_vnr'].value_counts()

0    535
1    343
Name: has_mas_vnr, dtype: int64

In [128]:
df['pool_qc'].value_counts()

No Pool    874
Ex           3
TA           1
Name: pool_qc, dtype: int64

In [129]:
df['has_pool'] = 0
df.loc[(df['pool_qc'] != 'No Pool'), 'has_pool'] = 1

In [130]:
df['has_pool'].value_counts()

0    874
1      4
Name: has_pool, dtype: int64

I'm interested in instantiating a model that takes into account the variables that show a correlation of above 0.5 or maybe 0.4, seeing what that looks like.

### Exploring Categorical Columns

In order to start grasping what's going on in the categorical columns, I wanted to get to a list of categorical column names. I created a data frame with the column names (as index) and data types. I used that to create a dataframe that consisted only of the categorical variables. I then turned the index for that into a list, my list of categorical columns, which I could then pass through the data frame to look at those a little more carefully.

NOTE: I returned up here repeatedly to clean out my categorical list when I recast things to numericals to make it easier to keep track of what I had left to work through.

In [131]:
column_dtype = pd.DataFrame(df.dtypes, df.columns)
column_dtype.rename(columns = {0: 'dtype'}, inplace = True)
column_dtype.dtypes

dtype    object
dtype: object

In [132]:
categorical_columns = column_dtype[(column_dtype['dtype'] != 'int64') & (column_dtype['dtype'] != 'float64')]


In [133]:
categorical_col_list = list(categorical_columns.index)
df[categorical_col_list].head()

,ms_zoning,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,...,garage_type,garage_yr_blt,garage_finish,garage_qual,garage_cond,paved_drive,pool_qc,fence,misc_feature,sale_type
0,RM,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,...,Detchd,1910.0,Unf,Po,Po,Y,No Pool,No Fence,No Addl Features,WD
1,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,...,Attchd,1977.0,Fin,TA,TA,Y,No Pool,No Fence,No Addl Features,WD
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,2006.0,RFn,TA,TA,Y,No Pool,No Fence,No Addl Features,New
3,RM,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,...,Detchd,1935.0,Unf,Fa,TA,N,No Pool,No Fence,No Addl Features,WD
4,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,1963.0,RFn,TA,TA,Y,No Pool,No Fence,No Addl Features,WD


#### A Note on how I organized my work.
I used the following code block to keep track of the categorical values I had left to decide how to deal with.

I took the list of details about the "has_(BLANK)" categories I created out in order to make the list more manageable. I left in categorical columns that I haven't used at all in the list for easy reference.

As I turned values into numericals (i.e. 0 = No, 1 = Yes; scaled ratings, etc.) and when I eliminated 'utilites', I took them out of the list to make it easier to keep track of which variables I needed to look at. 

As a result, I went back and forth to these next two blocks as I eliminated things I was working on, just to keep track.

Also, this [stackoverflow](https://stackoverflow.com/questions/52797035/how-to-remove-an-item-from-a-list-in-python-with-substring-match) gave me the idea to use a list comprehension to do this, though I varied the actual code.

In [134]:
clean_categorical_col_list = [i for i in categorical_col_list if 'bsmt' not in i and 'garage' not in i\
                             and 'utilities' not in i and 'alley' not in i and 'pool' not in i\
                             and 'fireplace' not in i and 'street' not in i and 'lot_shape' not in i\
                             and 'land_slope' not in i and 'exter' not in i and 'paved_drive' not in i\
                             and 'central_air' not in i and 'heating_qc' not in i and 'kitchen_qual' not in i\
                             and 'functional' not in i]
clean_categorical_col_list

['ms_zoning',
 'land_contour',
 'lot_config',
 'neighborhood',
 'condition_1',
 'condition_2',
 'bldg_type',
 'house_style',
 'roof_style',
 'roof_matl',
 'mas_vnr_type',
 'foundation',
 'heating',
 'electrical',
 'fence',
 'misc_feature',
 'sale_type']

In [135]:
df[clean_categorical_col_list].head()

,ms_zoning,land_contour,lot_config,neighborhood,condition_1,condition_2,bldg_type,house_style,roof_style,roof_matl,mas_vnr_type,foundation,heating,electrical,fence,misc_feature,sale_type
0,RM,Lvl,Inside,OldTown,Norm,Norm,2fmCon,2Story,Gable,CompShg,None,Stone,GasA,FuseP,No Fence,No Addl Features,WD
1,RL,Lvl,Inside,Sawyer,Norm,Norm,Duplex,1Story,Gable,CompShg,None,CBlock,GasA,SBrkr,No Fence,No Addl Features,WD
2,RL,Lvl,Inside,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,None,PConc,GasA,SBrkr,No Fence,No Addl Features,New
3,RM,Lvl,Inside,OldTown,Norm,Norm,1Fam,1Story,Gable,CompShg,None,CBlock,GasA,SBrkr,No Fence,No Addl Features,WD
4,RL,Lvl,Inside,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,BrkFace,CBlock,GasA,SBrkr,No Fence,No Addl Features,WD


In [136]:
df['has_alley'] = 0
df.loc[(df['alley'] != 'None'), 'has_alley'] = 1

In [137]:
df['has_alley'].value_counts()

0    820
1     58
Name: has_alley, dtype: int64

In [138]:
df['has_fireplace'] = 0
df.loc[(df['fireplace_qu'] != 'No Fireplace'), 'has_fireplace'] = 1
df['has_fireplace'].value_counts()

1    456
0    422
Name: has_fireplace, dtype: int64

-----

In [139]:
df['central_air'].value_counts()

Y    823
N     55
Name: central_air, dtype: int64

In [140]:
df['central_air'] = df['central_air'].map({'Y':1, 'N': 0})
df['central_air'].value_counts()

1    823
0     55
Name: central_air, dtype: int64

------

In [141]:
df['street'].value_counts()

Pave    873
Grvl      5
Name: street, dtype: int64

In [142]:
df.rename(columns = {'street':'street_surface'}, inplace = True)
df['street_surface'] = df['street_surface'].map({'Pave': 1, 'Grvl': 0})
df['street_surface'].head()

0    1
1    1
2    1
3    1
4    1
Name: street_surface, dtype: int64

In [143]:
df['paved_drive'].value_counts()

Y    790
N     65
P     23
Name: paved_drive, dtype: int64

In [144]:
df['paved_drive'] = df['paved_drive'].map({'N': 0, 'P': 1, 'Y': 2})

In [145]:
df['paved_drive'].value_counts()

2    790
0     65
1     23
Name: paved_drive, dtype: int64

------

In [146]:
df['lot_shape'].value_counts()

Reg    564
IR1    286
IR2     21
IR3      7
Name: lot_shape, dtype: int64

In [147]:
df['lot_shape'] = df['lot_shape'].map({'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3':3})
df['lot_shape'].value_counts()

0    564
1    286
2     21
3      7
Name: lot_shape, dtype: int64

In [148]:
df['land_slope'].value_counts()

Gtl    835
Mod     37
Sev      6
Name: land_slope, dtype: int64

In [149]:
df['land_slope'] = df['land_slope'].map({'Gtl':0, 'Mod':1, 'Sev':2})
df['land_slope'].value_counts()

0    835
1     37
2      6
Name: land_slope, dtype: int64

In [150]:
df['pool_qc'] = df['pool_qc'].map({'No Pool': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex':4})

In [151]:
df['pool_qc'].value_counts()

0    874
4      3
2      1
Name: pool_qc, dtype: int64

In [152]:
df['garage_qual'] = df['garage_qual'].map({'No Garage': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5})

In [153]:
df['garage_qual'].value_counts()

3    782
0     45
2     42
4      6
1      3
Name: garage_qual, dtype: int64

In [154]:
df['garage_cond'] = df['garage_cond'].map({'No Garage': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5})

In [155]:
df['garage_cond'].value_counts()

3    796
0     45
2     27
1      6
4      3
5      1
Name: garage_cond, dtype: int64

In [156]:
df['exter_cond'].value_counts()

TA    770
Gd     84
Fa     18
Ex      5
Po      1
Name: exter_cond, dtype: int64

In [157]:
df['exter_cond'] = df['exter_cond'].map({'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex':4})

In [158]:
df['exter_cond'].value_counts()

2    770
3     84
1     18
4      5
0      1
Name: exter_cond, dtype: int64

In [159]:
df['exter_qual'] = df['exter_qual'].map({'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex':4})

In [160]:
df['exter_qual'].value_counts()

2    552
3    292
4     25
1      9
Name: exter_qual, dtype: int64

In [161]:
df['heating_qc'].value_counts()

Ex    429
TA    267
Gd    157
Fa     25
Name: heating_qc, dtype: int64

In [162]:
df['heating_qc'] = df['heating_qc'].map({'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex':4})

In [163]:
df['heating_qc'].value_counts()

4    429
2    267
3    157
1     25
Name: heating_qc, dtype: int64

In [164]:
df['kitchen_qual'].value_counts()

TA    447
Gd    354
Ex     53
Fa     23
Po      1
Name: kitchen_qual, dtype: int64

In [165]:
df['kitchen_qual'] = df['kitchen_qual'].map({'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex':4})

In [166]:
df['kitchen_qual'].value_counts()

2    447
3    354
4     53
1     23
0      1
Name: kitchen_qual, dtype: int64

0 is Typical, 1 is Minor Deductions 1, 2 is Minor Deductions 2, 3 is Moderate Deductions, 4 is Major Deductions 1, 5 is Major Deductions 2, 6 is Severely Damaged, 7 is Salvage only.

In [167]:
df['functional'].value_counts()

Typ     812
Min2     28
Min1     23
Maj1      7
Mod       6
Maj2      2
Name: functional, dtype: int64

In [168]:
df['functional'] = df['functional'].map({'Typ': 0, 'Min1': 1, 'Min2': 2, 'Mod': 3, 'Maj1': 4, 
                                         'Maj2': 5, 'Sev': 6, 'Sal':7})

In [169]:
df['functional'].value_counts()

0    812
2     28
1     23
4      7
3      6
5      2
Name: functional, dtype: int64

-----

In [170]:
df['neighborhood'].value_counts()

NAmes      133
CollgCr     87
OldTown     76
Somerst     52
Edwards     50
Gilbert     49
NridgHt     44
NWAmes      44
Sawyer      40
SawyerW     38
Mitchel     32
Crawfor     32
BrkSide     32
Timber      24
IDOTRR      24
NoRidge     23
ClearCr     17
SWISU       16
MeadowV     13
StoneBr     13
BrDale      11
Veenker      7
Blmngtn      6
NPkVill      6
Greens       5
Blueste      4
Name: neighborhood, dtype: int64

In [171]:
df['ms_zoning'].value_counts()

RL         674
RM         146
FV          38
RH          13
C (all)      6
I (all)      1
Name: ms_zoning, dtype: int64

In [172]:
df['lot_config'].value_counts()

Inside     636
Corner     163
CulDSac     49
FR2         25
FR3          5
Name: lot_config, dtype: int64

In [173]:
df['land_contour'].value_counts()

Lvl    790
Bnk     36
HLS     35
Low     17
Name: land_contour, dtype: int64

In [174]:
df['condition_1'].value_counts()

Norm      755
Feedr      55
Artery     22
RRAn       14
PosN       11
PosA        8
RRAe        7
RRNn        3
RRNe        3
Name: condition_1, dtype: int64

In [175]:
df['bldg_type'].value_counts()

1Fam      724
TwnhsE     72
Duplex     34
Twnhs      32
2fmCon     16
Name: bldg_type, dtype: int64

In [176]:
df['house_style'].value_counts()

1Story    422
2Story    274
1.5Fin     96
SLvl       34
SFoyer     33
2.5Unf     10
1.5Unf      7
2.5Fin      2
Name: house_style, dtype: int64

In [177]:
df['roof_matl'].value_counts()

CompShg    861
Tar&Grv      8
WdShake      5
WdShngl      2
Metal        1
Roll         1
Name: roof_matl, dtype: int64

In [178]:
df['roof_style'].value_counts()

Gable      702
Hip        153
Gambrel     10
Flat         7
Mansard      4
Shed         2
Name: roof_style, dtype: int64

-----

In [179]:
df['utilities'].value_counts()

AllPub    877
NoSewr      1
Name: utilities, dtype: int64

In [180]:
df[df['utilities'] == 'NoSeWa']

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street_surface,alley,lot_shape,land_contour,...,mo_sold,yr_sold,sale_type,saleprice,has_bsmt,has_garage,has_mas_vnr,has_pool,has_alley,has_fireplace


In [181]:
df[df['utilities'] == 'NoSewr']

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street_surface,alley,lot_shape,land_contour,...,mo_sold,yr_sold,sale_type,saleprice,has_bsmt,has_garage,has_mas_vnr,has_pool,has_alley,has_fireplace
143,1014,527226020,20,RL,0.0,31220,1,None,1,Bnk,...,5,2008,WD,0,1,1,0,0,0,1


Deleting 'utilities'.

In [182]:
df.drop(columns = 'utilities', inplace = True)

In [183]:
df.shape

(878, 86)

In [184]:
df.to_csv('datasets/draft2_no_dummies_kaggle_test.csv', index = False)